In [ ]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data
import multiprocessing
from sklearn.metrics import accuracy_score

# Paths for image directory and model
EVAL_DIR = r'path to test folder'
EVAL_MODEL = r'path to model'

# Load the model for evaluation
model = torch.load(EVAL_MODEL)
model.eval()

# Configure batch size and number of CPUs
num_cpu = multiprocessing.cpu_count()
bs = 8

# Prepare the eval data loader
eval_transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5471, 0.4019, 0.3468],
                             [0.1961, 0.1704, 0.1652])])

eval_dataset = datasets.ImageFolder(root=EVAL_DIR, transform=eval_transform)
eval_loader = data.DataLoader(eval_dataset, batch_size=bs, shuffle=True,
                              num_workers=num_cpu, pin_memory=True)

# Enable GPU mode, if CUDA available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Class label names
class_names = eval_dataset.classes

# Initialize the prediction and label lists
predlist = torch.zeros(0, dtype=torch.long, device='cpu')
lbllist = torch.zeros(0, dtype=torch.long, device='cpu')

# Evaluate the model accuracy on the dataset
correct = 0
total = 0
with torch.no_grad():
    for images, labels in eval_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        predlist = torch.cat([predlist, predicted.view(-1).cpu()])
        lbllist = torch.cat([lbllist, labels.view(-1).cpu()])

# Overall accuracy
overall_accuracy = 100 * correct / total
print('Accuracy of the network on the {:d} test images: {:.2f}%'.format(len(eval_dataset), overall_accuracy))

# Accuracy for each class
correct_counts = np.zeros(len(class_names))
total_counts = np.zeros(len(class_names))

for true, pred in zip(lbllist.numpy(), predlist.numpy()):
    total_counts[true] += 1
    if true == pred:
        correct_counts[true] += 1

class_accuracies = 100 * correct_counts / total_counts
print('Per class accuracy')
print('-' * 18)
for class_name, accuracy in zip(class_names, class_accuracies):
    print('Accuracy of class {:<25}: {:.2f}%'.format(class_name, accuracy))

